In [1]:
!pip install contractions

import pandas as pd
import numpy as np
import json
import nest_asyncio
import asyncio
import re
import html
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import label_binarize
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, train_test_split
from keras.optimizers import Adam
from collections import Counter
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, roc_curve, auc, average_precision_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.sparse import hstack

2025-11-30 14:13:14.656342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764511995.030590      99 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764511995.130206      99 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
# read data
train = pd.read_json("../data/processed/IMDB_reviews_train_cleaned.json")
test = pd.read_json("../data/processed/IMDB_reviews_test.json")

Undersampling

In [3]:
# Check the original distribution of the combined groups
group_counts = train.groupby(['is_spoiler']).size()
print("Original joint counts:\n", group_counts)

# Determine the minimum and maximum size for balancing all groups
min_group_size = group_counts.min()
max_group_size = group_counts.max()
print(f"\nTarget minimum sample size per joint group: {min_group_size}")
print(f"\nTarget maximum sample size per joint group: {max_group_size}")

# Undersample each train group to the minimum size found
undersampled_train = train.groupby(['is_spoiler']).apply(
    lambda x: x.sample(n=min_group_size, replace=False, random_state=3244)
).reset_index(drop=True)

print("Undersampled Train shape:", undersampled_train.shape)
print("New joint counts:\n", undersampled_train.groupby(['is_spoiler']).size())

# Undersample each test group to the minimum size found
test_group_counts = test.groupby(['is_spoiler']).size()
test_min_group_size = test_group_counts.min()
print(f"\nTest minimum sample size per joint group: {test_min_group_size}")
undersampled_test = test.groupby(['is_spoiler']).apply(
    lambda x: x.sample(n=test_min_group_size, replace=False, random_state=3244)
).reset_index(drop=True)

print("Undersampled Test shape:", undersampled_test.shape)
print("New joint counts:\n", undersampled_test.groupby(['is_spoiler']).size())

Original joint counts:
 is_spoiler
False    338256
True     120868
dtype: int64

Target minimum sample size per joint group: 120868

Target maximum sample size per joint group: 338256


/tmp/ipykernel_99/2103583486.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  undersampled_train = train.groupby(['is_spoiler']).apply(


Undersampled Train shape: (241736, 21)
New joint counts:
 is_spoiler
False    120868
True     120868
dtype: int64

Test minimum sample size per joint group: 30056
Undersampled Test shape: (60112, 21)
New joint counts:
 is_spoiler
False    30056
True     30056
dtype: int64


/tmp/ipykernel_99/2103583486.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  undersampled_test = test.groupby(['is_spoiler']).apply(


Naive Bayes (naive approach)

In [9]:
# prepare data
undersampled_train["is_spoiler"] = undersampled_train["is_spoiler"].astype(int)
undersampled_test["is_spoiler"] = undersampled_test["is_spoiler"].astype(int)

train1_X = undersampled_train["review_text"].fillna("")
train1_y = undersampled_train["is_spoiler"]

test1_X = undersampled_test['review_text'].fillna("")
test1_y = undersampled_test['is_spoiler']

# vectorise
tfidf1 = TfidfVectorizer(max_features=150000, ngram_range=(1,3))
train1_X_tfidf = tfidf1.fit_transform(train1_X)
test1_X_tfidf = tfidf1.transform(test1_X)

# fitting model
m1 = MultinomialNB()
m1.fit(train1_X_tfidf, train1_y)

# prediction
y_pred1 = m1.predict(test1_X_tfidf)
y_pred_prob1 = m1.predict_proba(test1_X_tfidf)[:, 1]

In [10]:
# print evaluation metrices
acc1 = accuracy_score(test1_y, y_pred1)
prec1 = precision_score(test1_y, y_pred1)
rec1 = recall_score(test1_y, y_pred1)
f1_1 = f1_score(test1_y, y_pred1)
f2_1 = fbeta_score(test1_y, y_pred1, beta=2)
roc1 = roc_auc_score(test1_y, y_pred_prob1)
aps1 = average_precision_score(test1_y, y_pred_prob1)

print(f"Accuracy: {acc1:.4f}")
print(f"Precision: {prec1:.4f}")
print(f"Recall: {rec1:.4f}")
print(f"F1-score: {f1_1:.4f}")
print(f"F2-score: {f2_1:.4f}")
print(f"ROC AUC: {roc1:.4f}")
print(f"Average Precision (AP): {aps1:.4f}")
print(classification_report(test1_y, y_pred1))

Accuracy: 0.6727
Precision: 0.6732
Recall: 0.6711
F1-score: 0.6722
F2-score: 0.6716
ROC AUC: 0.7330
Average Precision (AP): 0.7315
              precision    recall  f1-score   support

           0       0.67      0.67      0.67     30056
           1       0.67      0.67      0.67     30056

    accuracy                           0.67     60112
   macro avg       0.67      0.67      0.67     60112
weighted avg       0.67      0.67      0.67     60112



In [11]:
confusion_matrix(test1_y, y_pred1)

array([[20264,  9792],
       [ 9884, 20172]])

Naive Bayes + Cosine Similarity

In [ ]:
undersampled_train["is_spoiler"] = undersampled_train["is_spoiler"].astype(int)
undersampled_test["is_spoiler"] = undersampled_test["is_spoiler"].astype(int)
undersampled_train["review_text"] = undersampled_train["review_text"].fillna("").astype(str)
undersampled_train["plot_synopsis"] = undersampled_train["plot_synopsis"].fillna("").astype(str)
undersampled_test["review_text"] = undersampled_test["review_text"].fillna("").astype(str)
undersampled_test["plot_synopsis"] = undersampled_test["plot_synopsis"].fillna("").astype(str)

# vectorise review_text and plot_synopsis 
tfidf2 = TfidfVectorizer(max_features=150000, ngram_range=(1,3))
temp = pd.concat([train['review_text'], train['plot_synopsis']], ignore_index=True)
tfidf2.fit(temp) # all vectors use the same vocab

train_rev = tfidf2.transform(undersampled_train['review_text'])
train_plot = tfidf2.transform(undersampled_train['plot_synopsis'])

test_rev = tfidf2.transform(undersampled_test['review_text'])
test_plot = tfidf2.transform(undersampled_test['plot_synopsis'])

# calculate cosine similiarity for train and test
train_cos = []
for i in range(len(train)):
    sim = cosine_similarity(train_rev[i], train_plot[i])[0][0]
    train_cos.append(sim)

train['cos_sim'] = train_cos

test_cos = []
for j in range(len(test)):
    sim = cosine_similarity(test_rev[j], test_plot[j])[0][0]
    test_cos.append(sim)

test['cos_sim'] = test_cos

# use both review_text and cosine_feature as features
train_cos_score = np.array(train['cos_sim']).reshape(-1, 1)
train2_X = hstack([train_rev, train_cos_score])
train2_y = train['is_spoiler']

test_cos_score = np.array(test['cos_sim']).reshape(-1, 1)
test2_X = hstack([test_rev, test_cos_score])
test2_y = test['is_spoiler']

# fitting model 
m2 = MultinomialNB()
m2.fit(train2_X, train2_y)

# prediction
y_pred2 = m2.predict(test2_X)
y_pred_prob2 = m2.predict_proba(test2_X)[:, 1]

In [ ]:
# print evaluation metrices
acc2 = accuracy_score(test2_y, y_pred2)
prec2 = precision_score(test2_y, y_pred2)
rec2 = recall_score(test2_y, y_pred2)
f1_2 = f1_score(test2_y, y_pred2)
f2_2 = fbeta_score(test2_y, y_pred2, beta=2)
roc2 = roc_auc_score(test2_y, y_pred_prob2)
aps2 = average_precision_score(test2_y, y_pred_prob2)

print(f"Accuracy: {acc2:.4f}")
print(f"Precision: {prec2:.4f}")
print(f"Recall: {rec2:.4f}")
print(f"F1-score: {f1_2:.4f}")
print(f"F2-score: {f2_2:.4f}")
print(f"ROC AUC: {roc2:.4f}")
print(f"Average Precision (AP): {aps2:.4f}")
print(classification_report(test2_y, y_pred2))

In [ ]:
confusion_matrix(test2_y, y_pred2)